# Minimal Transformer (PyTorch)

This notebook contains a minimal implementation of the Transformer architecture using PyTorch's autograd engine. It covers the core building blocks of attention-based models:
- Scaled Dot-Product Attention: compute attention weights from queries, keys, and values
- Multi-Head Attention: project into multiple subspaces and combine parallel attention heads
- Residual Connections and LayerNorm: stabilize training with skip connections and normalization
- Feed-Forward Network with GELU: apply non-linear transformations between attention layers
- Encoder and Decoder Blocks: stack layers to form the Transformer backbone